In [4]:
import os
import numpy as np
import pandas as pd
from mpc_utils import load_train_test_data_27s, load_train_test_data

In [5]:
x_train, y_train, x_test, y_test = load_train_test_data_27s(include_L=True)
l_train, l_test = x_train[:, :, 7:], x_test[:, :, 7:]
x_train, x_test = x_train[:, :, :7], x_test[:, :, :7]
x_train.shape, y_train.shape, x_test.shape, y_test.shape, l_train.shape

((200, 1350, 7), (200, 1350, 1), (25, 1350, 7), (25, 1350, 1), (200, 1350, 1))

In [6]:
from models.lmu_torch import LMUModel
import torch

def train_model(x_train, y_train, x_test, y_test, input_size, hidden_size, memory_size, theta, epochs=100, batch_size=16, lr=1e-3, device='cpu'):
    model = LMUModel(input_size=input_size, hidden_size=hidden_size, memory_size=memory_size, output_size=1, theta=theta, device=device)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss = torch.nn.MSELoss()
    n_batches = x_train.shape[0] // batch_size

    for epoch in range(epochs+1):
        # eval
        if epoch % 50 == 0:
            loss_train = []
            for batch_idx in range(n_batches-1):
                model.eval()
                x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
                y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
                x = torch.tensor(x, dtype=torch.float32, device=device)
                y = torch.tensor(y, dtype=torch.float32, device=device)
                ypr = model(x)
                loss_train.append(loss(ypr, y).item())
            loss_test = []
            for batch_idx in range(max(1, x_test.shape[0] // batch_size)):
                model.eval()
                x = x_test[batch_idx*batch_size:(batch_idx+1)*batch_size]
                y = y_test[batch_idx*batch_size:(batch_idx+1)*batch_size]
                x = torch.tensor(x, dtype=torch.float32, device=device)
                y = torch.tensor(y, dtype=torch.float32, device=device)
                ypr = model(x)
                loss_test.append(loss(ypr, y).item())
            print(epoch, 'train', np.array(loss_train).mean(), 'test', np.array(loss_test).mean())

        # train
        epoch_loss = []
        model.train()
        for batch_idx in range(n_batches-1):
            x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            x = torch.tensor(x, dtype=torch.float32, device=device)
            y = torch.tensor(y, dtype=torch.float32, device=device)

            optimizer.zero_grad()
            y_pred = model(x)
            l = loss(y_pred, y)
            l.backward()
            optimizer.step()

            epoch_loss.append(l.item())

        # log training loss
        avg_epoch_loss = np.array(epoch_loss).mean()
        if epoch % 10 == 0:
            print(epoch, avg_epoch_loss)
        else:
            print(epoch, avg_epoch_loss, end='\r')
    return model

In [4]:
hidden_size = 64
memory_size = 32
theta = 16

length_model = train_model(x_train, l_train, x_test, l_test, 7, hidden_size, memory_size, theta, epochs=100, batch_size=16, lr=1e-3)
model_name = f'mpc_models/lmu_length_7-1-{hidden_size}-{memory_size}-{theta}.pt'
torch.save(length_model.state_dict(), model_name)

0 train 0.1053507687015967 test 0.09588231891393661
0 0.09099637581543489
10 0.02335170233114199
20 0.012840221043337475
30 0.009863007136366585
40 0.009216473099182953
50 train 0.009070090695538303 test 0.01331960316747427
50 0.009083957496014509
60 0.009032332169061357
70 0.008989603766663508
80 0.008944488257508387
90 0.008893085643649101
100 train 0.008791436483575539 test 0.01313480269163847
100 0.008832149588587608


In [9]:
xtrl = np.concatenate([x_train, l_train], axis=2)
xtel = np.concatenate([x_test, l_test], axis=2)
action_model_true_length = train_model(xtrl, y_train, xtel, y_test, 8, hidden_size, memory_size, theta, epochs=250, batch_size=16, lr=1e-3)
model_name = f'mpc_models/lmu_action_true_7-1-{hidden_size}-{memory_size}-{theta}_250epochs.pt'
torch.save(action_model_true_length.state_dict(), model_name)

0 train 0.49338705973191693 test 0.4251982569694519
0 0.3003939932042902
10 0.030197971246459267
20 0.017432937398552895
30 0.013452758758582851
40 0.011286370117555965
50 train 0.009826862337914381 test 0.011015781201422215
50 0.009889701804654165
60 0.008871115955778143
70 0.008098993958397345
80 0.0075436259450560265
90 0.007089623715728521
100 train 0.006654849614609371 test 0.007684497162699699
100 0.006732017733156681
110 0.0064287441359324885
120 0.006165519145063378
130 0.0059427381916479635
140 0.0057501419108699665
150 train 0.005503616719083352 test 0.006301489192992449
150 0.005579321433536031
160 0.0054474276998503644
170 0.0052666694179854615
180 0.0050775121304799214
190 0.0049805663187395445
200 train 0.0049021320459856224 test 0.005426288582384586
200 0.004923061510040002
210 0.0047615041786974125
220 0.0046357327479530475
230 0.004675860918888991
240 0.004516785710372708
250 train 0.004872065418484536 test 0.005519535858184099
250 0.004748562583699822


In [14]:
hidden_size = 64
memory_size = 32
theta = 16

lmodel = LMUModel(input_size=7, output_size=1, hidden_size=hidden_size,
                  memory_size=memory_size, theta=theta)
lmodel.load_state_dict(torch.load('mpc_models/lmu_length_7-1-64-32-16.pt'))

amodel = LMUModel(input_size=8, output_size=1, hidden_size=hidden_size,
                  memory_size=memory_size, theta=theta)
amodel.load_state_dict(torch.load('mpc_models/lmu_action_true_7-1-64-32-16_250epochs.pt'))

<All keys matched successfully>

In [ ]:
# 0.007

In [6]:
import torch
from models.backpropamine_torch import BackpropamineRNN
from tqdm import tqdm

# model = LMUModel(input_size=7, output_size=1, hidden_size=hidden_size, 
#                     memory_size=memory_size, theta=theta)
model = BackpropamineRNN(isize=7, hsize=128, osize=1, freeze_plasticity=False)
optimizer = torch.optim.Adam(model.parameters())
loss = torch.nn.MSELoss()

epochs = 10
batch_size = 16

n_batches = x_train.shape[0] // batch_size

In [7]:
for epoch in range(epochs+1):

    hidden_hebb = model.initialZeroStateHebb(batch_size)

    epoch_loss = []
    model.train()
    for batch_idx in tqdm(range(n_batches-1)):
        hidden_hebb = model.initialZeroStateHebb(batch_size)
        x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
        y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        optimizer.zero_grad()

        y_pred = []
        for t in range(x.shape[1]):
            yp, hidden_hebb = model(x[:,t,:], hidden_hebb)
            y_pred.append(yp)
        y_pred = torch.stack(y_pred, dim=1)

        l = loss(y_pred, y)
        l.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss.append(l.item())

    avg_epoch_loss = np.array(epoch_loss).mean()
    if epoch % 10 == 0:
        print(epoch, avg_epoch_loss)
    else:
        print(epoch, avg_epoch_loss, end='\r')

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:45<00:00,  4.15s/it]


0 0.19694998792626642


100%|██████████| 11/11 [00:46<00:00,  4.18s/it]


100%|██████████| 11/11 [00:46<00:00,  4.22s/it]


...

100%|██████████| 11/11 [00:45<00:00,  4.15s/it]


100%|██████████| 11/11 [00:46<00:00,  4.23s/it]


100%|██████████| 11/11 [00:45<00:00,  4.18s/it]


100%|██████████| 11/11 [00:45<00:00,  4.16s/it]

10 0.012865105051208626


In [ ]:
input()

In [ ]:
import torch
import math
from torch import nn
from torch.nn import functional as F
from models.lmu_torch import leCunUniform
from scipy.signal import cont2discrete


class LMUCell(nn.Module):
    """ 
    LMU Cell

    Parameters:
        input_size (int) : 
            Size of the input vector (x_t)
        hidden_size (int) : 
            Size of the hidden vector (h_t)
        memory_size (int) :
            Size of the memory vector (m_t)
        theta (int) :
            The number of timesteps in the sliding window that is represented using the LTI system
        learn_a (boolean) :
            Whether to learn the matrix A (default = False)
        learn_b (boolean) :
            Whether to learn the matrix B (default = False)
    """

    def __init__(self, input_size, hidden_size, memory_size, theta, learn_a = False, learn_b = False):
        
        super(LMUCell, self).__init__()

        self.hidden_size = hidden_size
        self.memory_size = memory_size
        self.f = nn.Tanh()

        A, B = self.stateSpaceMatrices(memory_size, theta)
        A = torch.from_numpy(A).float()
        B = torch.from_numpy(B).float()

        if learn_a:
            self.A = nn.Parameter(A)
        else:
            self.register_buffer("A", A)
    
        if learn_b:
            self.B = nn.Parameter(B)
        else:
            self.register_buffer("B", B)

        # Declare Model parameters:
        ## Encoding vectors
        self.e_x = nn.Parameter(torch.empty(1, input_size))
        self.e_h = nn.Parameter(torch.empty(1, hidden_size))
        self.e_m = nn.Parameter(torch.empty(1, memory_size))
        ## Kernels
        self.W_x = nn.Parameter(torch.empty(hidden_size, input_size))
        self.W_h = nn.Parameter(torch.empty(hidden_size, hidden_size))
        self.W_m = nn.Parameter(torch.empty(hidden_size, memory_size))

        self.initParameters()

    def initParameters(self):
        """ Initialize the cell's parameters """

        # Initialize encoders
        leCunUniform(self.e_x)
        leCunUniform(self.e_h)
        nn.init.constant_(self.e_m, 0)
        # Initialize kernels
        nn.init.xavier_normal_(self.W_x)
        nn.init.xavier_normal_(self.W_h)
        nn.init.xavier_normal_(self.W_m)

    def stateSpaceMatrices(self, memory_size, theta):
        """ Returns the discretized state space matrices A and B """

        Q = np.arange(memory_size, dtype = np.float64).reshape(-1, 1)
        R = (2*Q + 1) / theta
        i, j = np.meshgrid(Q, Q, indexing = "ij")

        # Continuous
        A = R * np.where(i < j, -1, (-1.0)**(i - j + 1))
        B = R * ((-1.0)**Q)
        C = np.ones((1, memory_size))
        D = np.zeros((1,))

        # Convert to discrete
        A, B, C, D, dt = cont2discrete(
            system = (A, B, C, D), 
            dt = 1.0, 
            method = "zoh"
        )
        
        return A, B

    def forward(self, x, state):
        """
        Parameters:
            x (torch.tensor): 
                Input of size [batch_size, input_size]
            state (tuple): 
                h (torch.tensor) : [batch_size, hidden_size]
                m (torch.tensor) : [batch_size, memory_size]
        """

        h, m = state

        # Equation (7) of the paper
        u = F.linear(x, self.e_x) + F.linear(h, self.e_h) + F.linear(m, self.e_m) # [batch_size, 1]

        # Equation (4) of the paper
        m = F.linear(m, self.A) + F.linear(u, self.B) # [batch_size, memory_size]

        # Equation (6) of the paper
        h = self.f(
            F.linear(x, self.W_x) +
            F.linear(h, self.W_h) + 
            F.linear(m, self.W_m)
        ) # [batch_size, hidden_size]

        return h, m

# ------------------------------------------------------------------------------

class LMUModel(torch.nn.Module):
    """ A simple model for the psMNIST dataset consisting of a single LMU layer and a single dense classifier """

    def __init__(self, input_size, output_size, hidden_size, memory_size, theta, learn_a = False, learn_b = False):
        super(LMUModel, self).__init__()
        self.lmu = LMUCell(input_size, hidden_size, memory_size, theta, learn_a, learn_b)
        self.classifier = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = []
        h_0 = torch.zeros(x.shape[0], self.lmu.hidden_size)
        m_0 = torch.zeros(x.shape[0], self.lmu.memory_size)
        state = (h_0, m_0)
        for t in range(x.shape[1]):
            state = self.lmu(x[:,t,:], state) # [batch_size, hidden_size]
            output = self.classifier(state[0])
            out.append(output) # [batch_size, output_size]
        return torch.stack(out, dim=1) # [batch_size, seq_len, output_size]

class LinearHebbian(nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: torch.Tensor

    def __init__(self, in_features: int, out_features: int) -> None:
        factory_kwargs = {'device': None, 'dtype': None}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
        self.reset_parameters()

    def reset_parameters(self) -> None:
        # as in torch.nn.Linear
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
        nn.init.uniform_(self.bias, -bound, bound)
        # initialize the hebbian weights

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )